# Description

This notebook is the entry point for `run_notebook.py`
It is conceptually equivalent to `core/dataflow_model/master_experiment.py` for the `run_experiment.py` flow

This notebook:
- get a config from the environment
- create a DAG from the passed config
- run the DAG
- save the generated `ResultBundle`

In [1]:
%load_ext autoreload
%autoreload 2

import logging
import os

import core.config as cconfig
import dataflow as cdataf
import helpers.hdbg as dbg
import helpers.henv as henv
import helpers.hpickle as hpickl
import helpers.hprint as hprint

/Users/paul/src/amp/core/dataflow/models.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
dbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

# Packages
         python: 3.7.6
       gluonnlp: 0.9.1
        gluonts: 0.5.0
         joblib: 0.15.1
          mxnet: 1.6.0
          numpy: 1.17.5
         pandas: 1.0.3
        pyarrow: 0.17.1
          scipy: 1.4.1
        seaborn: 0.10.1
        sklearn: 0.23.1
    statsmodels: 0.11.1
# Last commits:
  * cad4359 paul     FRH4: initial check-in                                            (   2 hours ago) Mon Apr 19 14:15:20 2021  (HEAD -> frh4, origin/frh4)
  *   94a33e9 GP Saggese Merge pull request #153 from alphamatic/LemTask120_Use_usual_dev_approach_for_lm (   3 hours ago) Mon Apr 19 13:29:54 2021  (origin/master, origin/HEAD, master)
  |\  
  | * 8eaa80a saggese  Fix wrong references to IM                                        (   3 hours ago) Mon Apr 19 13:27:42 2021  (origin/LemTask120_Use_usual_dev_approach_for_lm)


In [3]:
config = cconfig.get_config_from_env()

In [4]:
dag_config = config.pop("DAG")

AttributeError: 'NoneType' object has no attribute 'pop'

In [ ]:
dag_runner = cdataf.PredictionDagRunner(dag_config, config["meta"]["dag_builder"])

In [ ]:
cdataf.draw(dag_runner.dag)

In [ ]:
if "set_fit_intervals" in config["meta"].to_dict():
    dag_runner.set_fit_intervals(
        **config["meta", "set_fit_intervals", "func_kwargs"].to_dict()
    )
if "set_predict_intervals" in config["meta"].to_dict():
    dag_runner.set_predict_intervals(
        **config["meta", "set_predict_intervals", "func_kwargs"].to_dict()
    )

In [ ]:
fit_result_bundle = dag_runner.fit()

In [ ]:
payload = cconfig.get_config_from_nested_dict({"config": config})

In [ ]:
if "run_oos" in config["meta"].to_dict().keys() and config["meta"]:
    result_bundle = dag_runner.predict()
    payload["fit_result_bundle"] = fit_result_bundle.to_config()
else:
    result_bundle = fit_result_bundle

In [ ]:
result_bundle.payload = payload

In [ ]:
# TODO(gp): Use  `cdtfut.save_experiment_result_bundle(config, result_bundle)`
try:
    path = os.path.join(
        config["meta", "experiment_result_dir"], "result_bundle.pkl"
    )
    if True:
        hpickl.to_pickle(result_bundle.to_config().to_dict(), path)
except AssertionError:
    _LOG.warning("Unable to serialize results.")